In [ ]:
import pandas as pd

df_orders = pd.read_csv("../datasets/olist_orders_dataset.csv")
df_order_items = pd.read_csv("../datasets/olist_order_items_dataset.csv")
df_customers = pd.read_csv("../datasets/olist_customers_dataset.csv")
df_geolocation = pd.read_csv("../datasets/olist_geolocation_dataset.csv")
df_payments = pd.read_csv("../datasets/olist_order_payments_dataset.csv")
df_reviews = pd.read_csv("../datasets/olist_order_reviews_dataset.csv")
df_products = pd.read_csv("../datasets/olist_products_dataset.csv")
df_sellers = pd.read_csv("../datasets/olist_sellers_dataset.csv")
df_category = pd.read_csv("../datasets/product_category_name_translation.csv")